# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv created with Weather assignment
weather = pd.read_csv('city_weather.csv')
weather

,Unnamed: 0,id,city,country,latitude,longitude,curr_weather,temperature,min_temperature,max_temperature,pressure,humidity%,visibility,cloudiness,wind_speed
0,0,3374210,sao filipe,São Filipe,14.8961,-24.4956,Clear,70.97,70.97,70.97,1015,74,10000,6,14.00
1,1,1014653,burgersdorp,Burgersdorp,-30.9977,26.3286,Clouds,67.10,67.10,67.10,1014,62,10000,55,7.20
2,2,4020109,atuona,Atuona,-9.8000,-139.0333,Clouds,78.76,78.76,78.76,1013,79,10000,80,20.47
3,3,6201424,mataura,Mataura,-46.1927,168.8643,Clear,59.49,59.49,59.49,1023,74,10000,10,1.12
4,4,3430443,necochea,Necochea,-38.5473,-58.7368,Clouds,78.04,78.04,78.04,1007,43,10000,35,13.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,576,6050066,la ronge,La Ronge,55.1001,-105.2842,Clear,3.45,1.62,3.45,1026,39,10000,0,2.30
577,577,2447513,arlit,Arlit,18.7369,7.3853,Clear,66.11,66.11,66.11,1016,10,10000,0,7.05
578,578,103630,najran,Najrān,17.4924,44.1277,Clear,66.85,66.85,66.85,1016,21,10000,3,2.37
579,579,3395458,maragogi,Maragogi,-9.0122,-35.2225,Clear,80.37,80.37,80.37,1013,71,10000,4,9.71


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather[["latitude", "longitude"]]

# Fill NaN values and convert to float
rating = weather["humidity%"].astype(int)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Filter data frame for ideal weather (maximum temperature <=80, minimum temperature  >= 70, humidity >=50)
# Drop rows with NAs

good_weather = weather.loc[(weather['max_temperature'] <= 80)& (weather['min_temperature'] >= 70)&(weather['humidity%']>=50)]
good_weather_df = good_weather.dropna(how='all', axis=1)
good_weather_df

,Unnamed: 0,id,city,country,latitude,longitude,curr_weather,temperature,min_temperature,max_temperature,pressure,humidity%,visibility,cloudiness,wind_speed
0,0,3374210,sao filipe,São Filipe,14.8961,-24.4956,Clear,70.97,70.97,70.97,1015,74,10000,6,14.00
2,2,4020109,atuona,Atuona,-9.8000,-139.0333,Clouds,78.76,78.76,78.76,1013,79,10000,80,20.47
24,24,2092164,lorengau,Lorengau,-2.0226,147.2712,Clouds,79.02,79.02,79.02,1008,79,10000,100,2.39
25,25,3366880,hermanus,Hermanus,-34.4187,19.2345,Clouds,70.81,70.81,73.90,1017,86,10000,97,11.56
26,26,2165478,gladstone,Gladstone,-23.8500,151.2500,Clouds,76.93,76.93,76.93,1004,78,10000,40,4.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,553,1731248,alugan,Alugan,12.2188,125.4808,Clouds,78.84,78.84,78.84,1014,77,10000,100,14.90
554,554,3347019,namibe,Mossamedes,-15.1961,12.1522,Rain,75.60,75.60,75.60,1013,85,10000,94,2.55
557,557,2173911,broken hill,Broken Hill,-31.9500,141.4333,Clear,74.91,74.91,74.91,1013,64,10000,2,10.36
559,559,2194098,ahipara,Ahipara,-35.1667,173.1667,Clear,73.13,73.13,73.13,1019,60,10000,4,4.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in good_weather_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        good_weather_df.loc[index, "Hotel Name"] = name_address["results"][1]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
#Rename dataframe as hotel_df
hotel_df = good_weather_df
hotel_df

,Unnamed: 0,id,city,country,latitude,longitude,curr_weather,temperature,min_temperature,max_temperature,pressure,humidity%,visibility,cloudiness,wind_speed,Hotel Name
0,0,3374210,sao filipe,São Filipe,14.8961,-24.4956,Clear,70.97,70.97,70.97,1015,74,10000,6,14.00,Hotel Xaguate
2,2,4020109,atuona,Atuona,-9.8000,-139.0333,Clouds,78.76,78.76,78.76,1013,79,10000,80,20.47,Banque Socredo
24,24,2092164,lorengau,Lorengau,-2.0226,147.2712,Clouds,79.02,79.02,79.02,1008,79,10000,100,2.39,Lorengau Harbourside Hotel
25,25,3366880,hermanus,Hermanus,-34.4187,19.2345,Clouds,70.81,70.81,73.90,1017,86,10000,97,11.56,Misty Waves Boutique Hotel
26,26,2165478,gladstone,Gladstone,-23.8500,151.2500,Clouds,76.93,76.93,76.93,1004,78,10000,40,4.61,Highpoint International Hotel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,553,1731248,alugan,Alugan,12.2188,125.4808,Clouds,78.84,78.84,78.84,1014,77,10000,100,14.90,Alugan Nature Water Refilling Station
554,554,3347019,namibe,Mossamedes,-15.1961,12.1522,Rain,75.60,75.60,75.60,1013,85,10000,94,2.55,Pavilhão Welwitschia Mirabilis
557,557,2173911,broken hill,Broken Hill,-31.9500,141.4333,Clear,74.91,74.91,74.91,1013,64,10000,2,10.36,Royal Exchange Hotel
559,559,2194098,ahipara,Ahipara,-35.1667,173.1667,Clear,73.13,73.13,73.13,1019,60,10000,4,4.21,Ahipara Holiday Park


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))